<a href="https://colab.research.google.com/github/Durixas/Parkinson-s-Disease-Prediction-Code-and-Data-Repository/blob/main/pre_agg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
roc_auc_score, roc_curve, confusion_matrix, cohen_kappa_score, confusion_matrix, matthews_corrcoef

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
import lightgbm as lgb
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
file_id = '1irvCs70iiI8WkXSZMMStmfPGfke5ivrP'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data.csv')

# Read the CSV file as DataFrame
import numpy as np
import pandas as pd

data = pd.read_csv('data.csv', header=1) # pd.read_excel() for .xlsx file

# Data Preprocessing

In [ ]:
data

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,250,0,0.80903,0.56355,0.28385,417,416,0.004627,0.000052,0.00064,...,3.0706,3.0190,3.1212,2.4921,3.5844,3.5400,3.3805,3.2003,6.8671,0
752,250,0,0.16084,0.56499,0.59194,415,413,0.004550,0.000220,0.00143,...,1.9704,1.7451,1.8277,2.4976,5.2981,4.2616,6.3042,10.9058,28.4170,0
753,251,0,0.88389,0.72335,0.46815,381,380,0.005069,0.000103,0.00076,...,51.5607,44.4641,26.1586,6.3076,2.8601,2.5361,3.5377,3.3545,5.0424,0
754,251,0,0.83782,0.74890,0.49823,340,339,0.005679,0.000055,0.00092,...,19.1607,12.8312,8.9434,2.2044,1.9496,1.9664,2.6801,2.8332,3.7131,0


In [ ]:
def split_data_by_id(data, test_size=0.3, random_state=None):
    unique_ids = data['id'].unique()

    train_ids, test_ids = train_test_split(unique_ids, test_size=test_size, random_state=random_state)

    train_data = data[data['id'].isin(train_ids)]
    test_data = data[data['id'].isin(test_ids)]

    return train_data, test_data

train_data, test_data = split_data_by_id(data, test_size=0.3, random_state=19260817)

groups = train_data.id

print("Train Data Shape:", train_data.shape)
print("Test Data Shape:", test_data.shape)

Train Data Shape: (528, 755)
Test Data Shape: (228, 755)


In [ ]:
X_train, y_train = train_data.drop(columns=['id']).iloc[:,:-1], train_data.iloc[:,-1]
X_test, y_test = test_data.drop(columns=['id']).iloc[:,:-1], test_data.iloc[:,-1]

In [ ]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    index=X_train.index,
    columns=X_train.columns
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    index=X_test.index,
    columns=X_test.columns
)
X_train, X_test = X_train_scaled, X_test_scaled

In [ ]:
train_data.iloc[:, 1:-1] = X_train

<ipython-input-10-872e2856be07>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3     -1.034692
4     -1.034692
5     -1.034692
6      0.966471
7      0.966471
         ...   
748    0.966471
749    0.966471
753   -1.034692
754   -1.034692
755   -1.034692
Name: gender, Length: 528, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.iloc[:, 1:-1] = X_train
<ipython-input-10-872e2856be07>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3     -1.426806
4     -0.870124
5     -0.966104
6     -0.044700
7     -0.083091
         ...   
748    0.128064
749    0.089672
753    0.521580
754    0.128064
755    0.128064
Name: numPulses, Length: 528, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_data.iloc[:, 1:-1] = X_train
<ipyt

In [ ]:
# Bag Over-sampling

random_state = 42
np.random.seed(random_state)

synthetic_data = train_data.copy()
grouped = synthetic_data.groupby('id')
centers = grouped.mean().iloc[:, :-1]
classes = grouped.first()['class']
minority_class_ids = centers[classes == 0].index
majority_class_ids = centers[classes == 1].index


minority_centers = centers.loc[minority_class_ids]
distances = cdist(minority_centers.values, minority_centers.values, metric='euclidean')
np.fill_diagonal(distances, np.inf)

K = 2
while len(minority_class_ids) <= len(majority_class_ids) - K:
    selected_idx = np.random.choice(minority_centers.index)
    selected_idx_row = minority_centers.index.get_loc(selected_idx)
    neighbor_indices = distances[selected_idx_row].argsort()[:2]
    neighbor_ids = minority_centers.index[neighbor_indices]

    for neighbor_id in neighbor_ids:
        original_group = grouped.get_group(selected_idx)
        neighbor_group = grouped.get_group(neighbor_id)

        new_id = data.id.max() + len(synthetic_data["id"].unique()) + 1
        for i in range(3):
            original_sample = original_group.sample(n=1, random_state=random_state + i).iloc[:, 1:-1].values
            neighbor_sample = neighbor_group.sample(n=1, random_state=random_state + i).iloc[:, 1:-1].values

            z = np.random.uniform(0, 1)
            new_sample_features = original_sample + z * (neighbor_sample - original_sample)
            new_sample = new_sample_features.squeeze().tolist() + [0]

            new_sample = pd.DataFrame([new_sample], columns=synthetic_data.columns[1:])
            new_sample['id'] = new_id

            synthetic_data = pd.concat([synthetic_data, new_sample], ignore_index=True)

    minority_class_ids = synthetic_data[synthetic_data['class'] == 0]['id'].unique()
synthetic_data

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,1,-1.034692,-2.039604,1.326383,0.732020,-1.426806,-1.420297,2.487074,-0.271986,0.723698,...,-0.534526,-0.591047,-0.527734,-0.019885,-0.458638,-0.473263,-0.628013,-0.580870,-0.791258,1
1,1,-1.034692,-2.546592,1.342106,0.290970,-0.870124,-0.864839,1.015330,3.254926,1.180666,...,-0.480345,-0.525732,-0.497033,-0.413890,-0.267058,-0.056289,-0.413387,-0.664775,-0.730831,1
2,1,-1.034692,-1.451798,1.193739,1.170593,-0.966104,-0.998915,0.725819,3.293964,2.157630,...,-0.511379,-0.547785,-0.462185,-0.402729,-0.405712,-0.310192,-0.719285,-0.772427,-0.815807,1
3,2,0.966471,0.088764,-1.178858,0.398241,-0.044700,-0.041228,-0.169520,-0.379055,-0.052358,...,2.035839,1.468668,0.469646,-0.321369,-0.518807,-0.477139,-0.118335,-0.055901,-0.682254,1
4,2,0.966471,0.549806,-1.188292,0.160049,-0.083091,-0.079535,-0.124287,-0.338662,0.184004,...,-0.532807,-0.352074,-0.452041,-0.480233,-0.489824,-0.397333,-0.187240,0.239907,-0.762325,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,512,-1.034692,0.547693,-0.384340,-1.115247,0.258876,0.261681,-0.869832,0.235810,-0.397713,...,-0.436079,-0.568872,-0.584505,-0.540623,-0.536454,-0.606293,-0.782489,-0.825451,-0.819845,0
782,512,-1.034692,0.471544,-0.457140,-1.810553,0.958334,0.959602,-0.965974,-0.490747,-0.720101,...,-0.165566,-0.551522,-0.617440,-0.546463,-0.537583,-0.569430,-0.678059,-0.741119,-0.815825,0
783,513,-1.034692,0.505524,-0.414595,-1.093837,0.636142,0.638118,-0.748299,-0.466429,-0.596849,...,0.270789,0.188955,0.451138,0.533276,0.249117,-0.073988,-0.417109,-0.567401,-0.771939,0
784,513,-1.034692,0.607636,-0.029357,-1.107666,0.689952,0.691810,-0.787165,-0.468266,-0.607443,...,-0.045184,-0.212793,-0.245317,-0.334252,-0.475429,-0.574737,-0.752089,-0.753467,-0.803238,0


In [ ]:
X_train, y_train = synthetic_data.drop(columns=['id']).iloc[:,:-1], synthetic_data.iloc[:,-1]
groups = synthetic_data.id

In [ ]:
X_train = X_train.groupby(X_train.index // 3).mean()
y_train = y_train.groupby(y_train.index // 3).mean()
X_test = X_test.groupby(X_test.index // 3).mean()
y_test = y_test.groupby(y_test.index // 3).mean()

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()

In [ ]:
# np.random.seed(42)

# pca = PCA()
# pca.fit(X_train)

# cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

# n_components = np.argmax(cumulative_variance >= 0.85) + 1

# pca = PCA(n_components=n_components)
# pca.fit(X_train)
# X_train_pca = pca.fit_transform(X_train)
# X_test_pca = pca.transform(X_test)

# print(f'Number of components: {n_components}')

# Model

In [ ]:
def print_grid_search(gs):
    print ("Best score: " + str(gs.best_score_))
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(best_parameters.keys()):
        print(param_name + ':' + str(best_parameters[param_name]))

def get_metrics(model, mode = 'test'):
    if mode == 'train':
        X, y = X_train, y_train
    elif mode == 'test':
        X, y = X_test, y_test
    results = {}
    y_pred = model.predict(X)
    y_pred_prob = model.predict_proba(X)[:, 1]

    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
    specificity = tn / (tn + fp)

    results['Accuracy'] = accuracy_score(y, y_pred)
    results['Precision'] = precision_score(y, y_pred)
    results['Recall'] = recall_score(y, y_pred)
    results['Specificity'] = specificity
    results['F1'] = f1_score(y, y_pred)
    # results['kappa'] = cohen_kappa_score(y, y_pred)
    results['AUC'] = roc_auc_score(y, y_pred_prob)
    results['MCC'] = matthews_corrcoef(y, y_pred)
    return results

def get_results(model):
    res1, res2 = get_metrics(model, mode='train'), get_metrics(model, mode='test')
    print(f'Train:\n{res1}')
    print(f'Test:\n{res2}')
    return res1, res2

def get_coma(model):
    X, y = X_test, y_test
    y_pred = model.predict(X)
    return confusion_matrix(y, y_pred)

## Logistic Regression

In [ ]:
params = {
    'C': [0.01, 0.1, 1.0],
    'penalty': ['l1', 'l2']
}
grid_lr = GridSearchCV(LogisticRegression(solver='liblinear',random_state=42), params, cv=5)
grid_lr.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=LogisticRegression(random_state=42, solver='liblinear'),
             param_grid={'C': [0.01, 0.1, 1.0], 'penalty': ['l1', 'l2']})

In [ ]:
print_grid_search(grid_lr)

Best score: 0.8820029027576197
Best parameters set:
C:1.0
penalty:l1


In [ ]:
best_lr = grid_lr.best_estimator_
res_lr_train, res_lr_test = get_results(best_lr)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8421052631578947, 'Precision': 0.8688524590163934, 'Recall': 0.9298245614035088, 'Specificity': 0.5789473684210527, 'F1': 0.8983050847457628, 'AUC': 0.8217913204062789, 'MCC': 0.5535117231700726}


In [ ]:
df_train_result = pd.DataFrame(res_lr_train, index=['Logistic Regression'])
df_test_result = pd.DataFrame(res_lr_test, index=['Logistic Regression'])

## Decision Tree

In [ ]:
params = {
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_dt = GridSearchCV(DecisionTreeClassifier(random_state=42), params, cv=5)
grid_dt.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [ ]:
print_grid_search(grid_dt)

Best score: 0.8212626995645864
Best parameters set:
criterion:entropy
min_samples_leaf:1
min_samples_split:2


In [ ]:
best_dt = grid_dt.best_estimator_
res_dt_train, res_dt_test = get_results(best_dt)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.6973684210526315, 'Precision': 0.7833333333333333, 'Recall': 0.8245614035087719, 'Specificity': 0.3157894736842105, 'F1': 0.8034188034188035, 'AUC': 0.5701754385964912, 'MCC': 0.14907119849998599}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_dt_train, index=['Decision Tree'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_dt_test, index=['Decision Tree'])], axis=0)

## Random Forest

In [ ]:
params = {
    'n_estimators': [100, 200, 300],
    'max_features': ['log2', 'sqrt', 1]
}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), params, cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_features': ['log2', 'sqrt', 1],
                         'n_estimators': [100, 200, 300]})

In [ ]:
print_grid_search(grid_rf)

Best score: 0.9200290275761974
Best parameters set:
max_features:sqrt
n_estimators:200


In [ ]:
best_rf = grid_rf.best_estimator_
res_rf_train, res_rf_test = get_results(best_rf)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8421052631578947, 'Precision': 0.8571428571428571, 'Recall': 0.9473684210526315, 'Specificity': 0.5263157894736842, 'F1': 0.9, 'AUC': 0.8508771929824561, 'MCC': 0.5447047794019222}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_rf_train, index=['Random Forest'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_rf_test, index=['Random Forest'])], axis=0)

## Gradient Boosting

In [ ]:
params = {
    'n_estimators': [800],
    'learning_rate': [0.01, 0.02],
    'max_depth': [3, 5],
    'max_features': ['sqrt', 'log2']
}
grid_gb = GridSearchCV(GradientBoostingClassifier(random_state=42), params, cv=5)
grid_gb.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': [0.01, 0.02], 'max_depth': [3, 5],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [800]})

In [ ]:
print_grid_search(grid_gb)

Best score: 0.943033381712627
Best parameters set:
learning_rate:0.02
max_depth:5
max_features:sqrt
n_estimators:800


In [ ]:
best_gb = grid_gb.best_estimator_
res_gb_train, res_gb_test = get_results(best_gb)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8421052631578947, 'Precision': 0.8571428571428571, 'Recall': 0.9473684210526315, 'Specificity': 0.5263157894736842, 'F1': 0.9, 'AUC': 0.8753462603878116, 'MCC': 0.5447047794019222}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_gb_train, index=['GBDT'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_gb_test, index=['GBDT'])], axis=0)

## XGBoost

In [ ]:
params = {
    'n_estimators': [1000],
    'learning_rate': [0.01],
    'max_depth': [3],
    'min_child_weight': [1],
    'gamma': [0, 0.1],
    'subsample': [0.9],
    'colsample_bytree': [0.3],
}
grid_xgb = GridSearchCV(xgb.XGBClassifier(random_state=42), params, cv=5)
grid_xgb.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'colsample_bytree': [0.3], 'gamma': [0, 0.1],
                         'learning_rate': [0.01], 'max_depth': [3],
                         'min_child_weight': [1], 'n_estimators': [1000],
                         'subsample': [0.9]})

In [ ]:
print_grid_search(grid_xgb)

Best score: 0.9087082728592163
Best parameters set:
colsample_bytree:0.3
gamma:0
learning_rate:0.01
max_depth:3
min_child_weight:1
n_estimators:1000
subsample:0.9


In [ ]:
best_xgb = grid_xgb.best_estimator_
res_xgb_train, res_xgb_test = get_results(best_xgb)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8289473684210527, 'Precision': 0.8548387096774194, 'Recall': 0.9298245614035088, 'Specificity': 0.5263157894736842, 'F1': 0.8907563025210085, 'AUC': 0.8698060941828255, 'MCC': 0.5095101710852533}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_xgb_train, index=['XGBoost'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_xgb_test, index=['XGBoost'])], axis=0)

## LightGBM

In [ ]:
params = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.2, 0.3, 0.4],
    'max_depth': [3],
    'num_leaves': [10],
    'subsample': [0.3, 0.5],
    'colsample_bytree': [0.7, 0.8, 0.9]
}
grid_lgb = GridSearchCV(lgb.LGBMClassifier(n_jobs=-1, verbose=-1), params, cv=5)
grid_lgb.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LGBMClassifier(n_jobs=-1, verbose=-1),
             param_grid={'colsample_bytree': [0.7, 0.8, 0.9],
                         'learning_rate': [0.2, 0.3, 0.4], 'max_depth': [3],
                         'n_estimators': [50, 100, 150], 'num_leaves': [10],
                         'subsample': [0.3, 0.5]})

In [ ]:
print_grid_search(grid_lgb)

Best score: 0.9125544267053701
Best parameters set:
colsample_bytree:0.7
learning_rate:0.2
max_depth:3
n_estimators:150
num_leaves:10
subsample:0.3


In [ ]:
best_lgb = grid_lgb.best_estimator_
res_lgb_train, res_lgb_test = get_results(best_lgb)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8289473684210527, 'Precision': 0.8548387096774194, 'Recall': 0.9298245614035088, 'Specificity': 0.5263157894736842, 'F1': 0.8907563025210085, 'AUC': 0.8790397045244691, 'MCC': 0.5095101710852533}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_lgb_train, index=['LightGBM'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_lgb_test, index=['LightGBM'])], axis=0)

## KNN

In [ ]:
params = {
    'n_neighbors': [3, 4, 5],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}
grid_knn = GridSearchCV(KNeighborsClassifier(n_jobs=-1, algorithm='ball_tree'), params, cv=5)
grid_knn.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=KNeighborsClassifier(algorithm='ball_tree', n_jobs=-1),
             param_grid={'n_neighbors': [3, 4, 5], 'p': [1, 2],
                         'weights': ['uniform', 'distance']})

In [ ]:
print_grid_search(grid_knn)

Best score: 0.8208272859216255
Best parameters set:
n_neighbors:4
p:1
weights:distance


In [ ]:
best_knn = grid_knn.best_estimator_
res_knn_train, res_knn_test = get_results(best_knn)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.7105263157894737, 'Precision': 0.8723404255319149, 'Recall': 0.7192982456140351, 'Specificity': 0.6842105263157895, 'F1': 0.7884615384615384, 'AUC': 0.7691597414589104, 'MCC': 0.35968247729147257}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_knn_train, index=['KNN'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_knn_test, index=['KNN'])], axis=0)

## SVM

In [ ]:
params = {
    'C': [0.5, 1, 10],
    'gamma': ['scale', 'auto'],
}
grid_svm = GridSearchCV(SVC(random_state=42, probability=True), params, cv=5)
grid_svm.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(probability=True, random_state=42),
             param_grid={'C': [0.5, 1, 10], 'gamma': ['scale', 'auto']})

In [ ]:
print_grid_search(grid_svm)

Best score: 0.9354136429608128
Best parameters set:
C:10
gamma:scale


In [ ]:
best_svm = grid_svm.best_estimator_
res_svm_train, res_svm_test = get_results(best_svm)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8289473684210527, 'Precision': 0.84375, 'Recall': 0.9473684210526315, 'Specificity': 0.47368421052631576, 'F1': 0.8925619834710744, 'AUC': 0.7830101569713758, 'MCC': 0.5}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_svm_train, index=['SVM'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_svm_test, index=['SVM'])], axis=0)

## Naive Bayes

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)

GaussianNB()

In [ ]:
res_nb_train, res_nb_test = get_results(nb)

Train:
{'Accuracy': 0.8435114503816794, 'Precision': 0.8688524590163934, 'Recall': 0.8091603053435115, 'Specificity': 0.8778625954198473, 'F1': 0.8379446640316206, 'AUC': 0.8949653283608182, 'MCC': 0.6886500387556949}
Test:
{'Accuracy': 0.6710526315789473, 'Precision': 0.82, 'Recall': 0.7192982456140351, 'Specificity': 0.5263157894736842, 'F1': 0.7663551401869159, 'AUC': 0.7331486611265005, 'MCC': 0.224179415327122}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_nb_train, index=['Naive Bayes'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_nb_test, index=['Naive Bayes'])], axis=0)

## AdaBoost

In [ ]:
best_ada = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1),
    n_estimators=100,
    learning_rate=0.5,
    random_state=42
).fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [ ]:
res_ada_train, res_ada_test = get_results(best_ada)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8157894736842105, 'Precision': 0.8524590163934426, 'Recall': 0.9122807017543859, 'Specificity': 0.5263157894736842, 'F1': 0.8813559322033898, 'AUC': 0.8439519852262234, 'MCC': 0.47716527859489016}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_ada_train, index=['AdaBoost'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_ada_test, index=['AdaBoost'])], axis=0)

## MLP

In [ ]:
mlp_model = MLPClassifier(random_state=42, hidden_layer_sizes=(512,256,128), learning_rate_init=0.001)
mlp_model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(512, 256, 128), random_state=42)

In [ ]:
res_mlp_train, res_mlp_test = get_results(mlp_model)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.9078947368421053, 'Precision': 0.890625, 'Recall': 1.0, 'Specificity': 0.631578947368421, 'F1': 0.9421487603305785, 'AUC': 0.8707294552169897, 'MCC': 0.75}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_mlp_train, index=['MLP'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_mlp_test, index=['MLP'])], axis=0)

## Stacking

In [ ]:
estimators = [
    ('knn', KNeighborsClassifier(n_jobs=-1, algorithm='ball_tree')),
    ('svc', SVC(probability=True, gamma='auto', random_state=42))
]
final_estimator = LogisticRegression(solver='liblinear', random_state=42)

stacking_clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)

params = {
    'knn__n_neighbors': [2, 3, 4],
    'svc__C': [0.1, 1, 10],
    'final_estimator__C': [1, 10],
    'final_estimator__penalty': ['l1', 'l2']
}
grid_stack = GridSearchCV(estimator=stacking_clf, param_grid=params, cv=5, n_jobs=-1)
grid_stack.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=StackingClassifier(estimators=[('knn',
                                                       KNeighborsClassifier(algorithm='ball_tree',
                                                                            n_jobs=-1)),
                                                      ('svc',
                                                       SVC(gamma='auto',
                                                           probability=True,
                                                           random_state=42))],
                                          final_estimator=LogisticRegression(random_state=42,
                                                                             solver='liblinear')),
             n_jobs=-1,
             param_grid={'final_estimator__C': [1, 10],
                         'final_estimator__penalty': ['l1', 'l2'],
                         'knn__n_neighbors': [2, 3, 4],
                         'svc__C': [0.1, 1, 10]})

In [ ]:
print_grid_search(grid_stack)

Best score: 0.9317126269956459
Best parameters set:
final_estimator__C:1
final_estimator__penalty:l2
knn__n_neighbors:2
svc__C:10


In [ ]:
best_stack = grid_stack.best_estimator_
res_stack_train, res_stack_test = get_results(best_stack)

Train:
{'Accuracy': 1.0, 'Precision': 1.0, 'Recall': 1.0, 'Specificity': 1.0, 'F1': 1.0, 'AUC': 1.0, 'MCC': 1.0}
Test:
{'Accuracy': 0.8552631578947368, 'Precision': 0.8484848484848485, 'Recall': 0.9824561403508771, 'Specificity': 0.47368421052631576, 'F1': 0.9105691056910569, 'AUC': 0.8088642659279779, 'MCC': 0.5843065474681431}


In [ ]:
df_train_result = pd.concat([df_train_result, pd.DataFrame(res_stack_train, index=['Stacking'])], axis=0)
df_test_result = pd.concat([df_test_result, pd.DataFrame(res_stack_test, index=['Stacking'])], axis=0)

# Evaluation

In [ ]:
df_test_result_sorted = df_test_result.sort_values(by='Accuracy', ascending=False)
df_train_result_reordered = df_train_result.reindex(df_test_result_sorted.index)
print('Train:')
df_train_result_reordered

Train:


,Accuracy,Precision,Recall,Specificity,F1,AUC,MCC
MLP,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
Stacking,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
Logistic Regression,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
Random Forest,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
GBDT,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
XGBoost,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
LightGBM,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
SVM,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
AdaBoost,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000
KNN,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000


In [ ]:
print('Test:')
df_test_result_sorted

Test:


,Accuracy,Precision,Recall,Specificity,F1,AUC,MCC
MLP,0.907895,0.890625,1.000000,0.631579,0.942149,0.870729,0.750000
Stacking,0.855263,0.848485,0.982456,0.473684,0.910569,0.808864,0.584307
Logistic Regression,0.842105,0.868852,0.929825,0.578947,0.898305,0.821791,0.553512
Random Forest,0.842105,0.857143,0.947368,0.526316,0.900000,0.850877,0.544705
GBDT,0.842105,0.857143,0.947368,0.526316,0.900000,0.875346,0.544705
XGBoost,0.828947,0.854839,0.929825,0.526316,0.890756,0.869806,0.509510
LightGBM,0.828947,0.854839,0.929825,0.526316,0.890756,0.879040,0.509510
SVM,0.828947,0.843750,0.947368,0.473684,0.892562,0.783010,0.500000
AdaBoost,0.815789,0.852459,0.912281,0.526316,0.881356,0.843952,0.477165
KNN,0.710526,0.872340,0.719298,0.684211,0.788462,0.769160,0.359682


In [ ]:
df_train_result_reordered.to_csv('pre_mean.csv', index=True)
df_test_result_sorted.to_csv('pre_mean.csv', index=True)